In [5]:
import xnat
import zipfile
#import mysql.connector
#import json
import os
import pydicom as dicom

In [6]:
# set up XNAT login credentials here
# .......................
xnatUrl = 'http://172.23.0.5:80'  #change me! e.g. 'http://localhost:8081/'
# - check the url using docker network inspect on the package defaul network, find the web application nginx
#xnatUrl = 'https://xnat.bmia.nl'
xnatUser = 'admin'  #change me!
#xnatUser = 'leonardwee'  #change me!
xnatPass = 'admin'  #change me!
xnatProject = 'hn3-radiomics'  #change me!

In [7]:
# ----------------------------------------------------------------------------------- create a zip function
#fileList : a python list containing the path to each file to put packed into the zip archive
def createZip(fileList, fileName, omitPrefixFolder=None):
    fileZip = zipfile.ZipFile(fileName, 'w', zipfile.ZIP_DEFLATED)

    for myFile in fileList:
        archFileName = myFile
        if omitPrefixFolder is not None:
            archFileName = myFile.replace(omitPrefixFolder, '')
        try:
            fileZip.write(myFile, archFileName)
        except:
            print("Could not find file: " + myFile)

    fileZip.close()

    return fileName

In [8]:
# ----------------------------------------------------------------------------------- create an xnat upload function
#conn : is the connector handle to an XNAT service
#zipFilePath : is the fullPath to the zipped dicom data object
#project : is the unique identifier of the xnat project - see your setting for xnatProject above
#subject : is the unique identifier of the patient
#studyDescription : is the identifier of the study eg "CT"
def uploadData(conn, zipFilePath, project, subject, studyDescription):
    conn.services.import_(zipFilePath, content_type='application/zip',
                          project=project,
                          subject=subject,
                          experiment=studyDescription, overwrite="delete")

In [9]:
#script starts here

# Connecting to XNAT service
connection = xnat.connect(xnatUrl, user=xnatUser, password=xnatPass)

dataLocation = './data/xnat_import' #change me!

patientList = [ f.path for f in os.scandir(dataLocation) if f.is_dir() ]

for ff in patientList:
    patid = ff.split('/')[-1]
    targetFile = dataLocation+'/'+patid+'.zip' ##change me!
    listOfFiles = [os.path.join(dp, f) for dp, dn, filenames in os.walk(ff) for f in filenames]
    #this = dicom.read_file(listOfFiles[0], force=True)
    xnatSubject = patid
    xnatStudyDescription = 'CT' #or change to this.StudyDescription
    zipFileName = createZip(listOfFiles,targetFile)
    print(zipFileName)
    try:
        #uploadData(connection, zipFileName,xnatProject,xnatSubject,xnatStudyDescription)
        prearchive_session = connection.services.import_(zipFileName, project=xnatProject, destination='/prearchive')
        os.remove(zipFileName)
    except:
        print("Could not upload study : "+xnatSubject)

# Close XNAT connection
connection.disconnect()

print('FINISHED!')

[WARNING] Detected a redirect from http://172.23.0.5:80 to http://172.23.0.5/, using http://172.23.0.5/ from now on


./data/xnat_import/p0415c0006i4378200.zip
./data/xnat_import/p0415c0006i4441159.zip
./data/xnat_import/p0415c0006i4450765.zip
./data/xnat_import/p0415c0006i4461422.zip
./data/xnat_import/p0415c0006i4534464.zip
./data/xnat_import/p0415c0006i4641582.zip
./data/xnat_import/p0415c0006i4676602.zip
./data/xnat_import/p0415c0006i4798901.zip
./data/xnat_import/p0415c0006i4854737.zip
./data/xnat_import/p0415c0006i5035509.zip
./data/xnat_import/p0415c0006i5064021.zip
./data/xnat_import/p0415c0006i5120774.zip
./data/xnat_import/p0415c0006i5134193.zip
./data/xnat_import/p0415c0006i5217471.zip
./data/xnat_import/p0415c0006i5351827.zip
./data/xnat_import/p0415c0006i5424843.zip
./data/xnat_import/p0415c0006i5443091.zip
./data/xnat_import/p0415c0006i5461698.zip
./data/xnat_import/p0415c0006i5495649.zip
./data/xnat_import/p0415c0006i5678481.zip
./data/xnat_import/p0415c0006i5691192.zip
./data/xnat_import/p0415c0006i5715555.zip
./data/xnat_import/p0415c0006i5745399.zip
./data/xnat_import/p0415c0006i5770